In [1]:
def load_ejs_dataframe(url, name='ej_data', concat_with=[], subset=None):
    import pandas as pd 

    ## thanks to minyechil for this code fix to a version behavior difference
    try:
        df = pd.read_json(url, orient='split')
    except:
        import requests

        with requests.get(url) as r:
            if r.status_code == 200:
                data = r.json()['data']
                df = pd.DataFrame(data)
            else:
                return
                
    df_ret   = pd.concat([
    df.demographics.combine_first(
        df.main.combine_first(
            df.extras))
    ], axis=1)
    
    df_ret = df_ret.rename(columns={'demographics': name})
    
    if subset:
        df_ret = [[subset]]
    
    if concat_with:
        concat_with.append(df_ret)
        return pd.concat(concat_with, axis=1)
    else:
        return df_ret

## EXAMPLE TESTING

In [2]:
url = 'https://ejscreen.epa.gov/mapper/ejscreenRESTbroker1.aspx?namestr=&geometry={"spatialReference":{"wkid":4326},"rings":[[[-77.08493677593843,38.826233476701],[-77.00120416519032,38.826252459704506],[-77.00118490484675,38.77467231699507],[-77.08491751559487,38.77465332024619],[-77.08493677593843,38.826233476701]]]}&distance=&unit=9035&areatype=&areaid=&f=json'
d_ejs = load_ejs_dataframe(url)
d_ejs

,ej_data
P_ENGLISH,81
P_SPANISH,9
P_FRENCH,1
P_RUS_POL_SLAV,1
P_OTHER_IE,2
...,...
N_CG_NOHINCPCT_PCTILE,36
N_CI_FLOOD_PCTILE,81
N_CI_FLOOD30_PCTILE,80
N_CI_FIRE_PCTILE,0


## API DATA LOADING

In [3]:
# westlake 
url = 'https://ejscreen.epa.gov/mapper/ejscreenRESTbroker1.aspx?namestr=&geometry={"spatialReference":{"wkid":4326},"rings":[[[-93.2653374237029,30.258431907875167],[-93.23881574096369,30.258431907875167],[-93.23881574096369,30.235965757985912],[-93.2653374237029,30.235965757985912],[-93.2653374237029,30.258431907875167]]]}&distance=&unit=9035&areatype=&areaid=&f=json'
d_ejs = load_ejs_dataframe(url, 'westlake')
d_ejs

# mossville
url = 'https://ejscreen.epa.gov/mapper/ejscreenRESTbroker1.aspx?namestr=&geometry={"spatialReference":{"wkid":4326},"rings":[[[-93.30241628112472,30.258431907875167],[-93.27228970946949,30.258431907875167],[-93.27228970946949,30.236262374278827],[-93.30241628112472,30.236262374278827],[-93.30241628112472,30.258431907875167]]]}&distance=&unit=9035&areatype=&areaid=&f=json'
d_ejs = load_ejs_dataframe(url, 'mossville', concat_with=[d_ejs])
d_ejs

# lake charles
url = 'https://ejscreen.epa.gov/mapper/ejscreenRESTbroker1.aspx?namestr=&geometry={"spatialReference":{"wkid":4326},"rings":[[[-93.21984715881035,30.233963574598853],[-93.1872314971893,30.233963574598853],[-93.1872314971893,30.208154006134215],[-93.21984715881035,30.208154006134215],[-93.21984715881035,30.233963574598853]]]}&distance=&unit=9035&areatype=&areaid=&f=json'
d_ejs = load_ejs_dataframe(url, 'lake_charles', concat_with=[d_ejs])
d_ejs

,westlake,mossville,lake_charles
P_ENGLISH,100.0,N/A,96.0
P_SPANISH,0.0,N/A,2.0
P_FRENCH,0.0,N/A,1.0
P_RUS_POL_SLAV,0.0,N/A,0.0
P_OTHER_IE,0.0,N/A,1.0
...,...,...,...
N_CG_NOHINCPCT_PCTILE,32,34,72
N_CI_FLOOD_PCTILE,70,81,82
N_CI_FLOOD30_PCTILE,78,81,84
N_CI_FIRE_PCTILE,82,78,0


## EJ SCREEN SUBSETTING

In [4]:
aq_ej = \
    ['RAW_E_PM25',
     'S_E_PM25',
     'S_E_PM25_PER',
     'S_P2_PM25',
     'S_P5_PM25',
     'N_E_PM25',
     'N_E_PM25_PER',
     'N_P2_PM25',
     'N_P5_PM25',
     'totalPop',
     'RAW_E_RSEI_AIR', 
     'PCT_MINORITY', 
     'RAW_HI_CANCER',
     'RAW_HI_LIFEEXP',
     'P_NHBLACK',
     'NUM_AIRPOLL',
     'NUM_TRI']

In [5]:
d_ejs.loc[aq_ej]

,westlake,mossville,lake_charles
RAW_E_PM25,8.67,8.7,8.56
S_E_PM25,8.28,8.28,8.28
S_E_PM25_PER,88,89,84
S_P2_PM25,61,79,90
S_P5_PM25,78,87,88
N_E_PM25,8.45,8.45,8.45
N_E_PM25_PER,66,67,63
N_P2_PM25,57,74,85
N_P5_PM25,72,81,82
totalPop,2540,209,11080
